In [6]:
import pandas as pd
import string
import re
import time

In [8]:
df = pd.read_parquet(r"cleaned_test.parquet")

In [9]:
len(df)

23767

In [ ]:
len(text)

In [118]:
import pandas as pd
import string
import re
import time
from collections import Counter
from collections import defaultdict
import json
from tqdm import tqdm

allowed_chars = set(string.ascii_letters + string.digits + string.punctuation + " \n\t")

# df = pd.read_parquet(r'wikitext\wikitext-103-raw-v1\train-00000-of-00002.parquet')
# df = df[df['text'].str.strip() != ""].reset_index(drop=True)
# df['text'] = df['text'].apply(lambda x: ''.join(c for c in x if c in allowed_chars))

# def fix_spaces(text):
#     return re.sub(r'\s+([,.!?;:])', r'\1', text)

# df["text"] = df["text"].apply(fix_spaces)
# df["text"] = df["text"].str.replace(" @-@ ", "-", regex=False)


# df.to_parquet("cleaned_test.parquet")

# df = pd.read_parquet("cleaned_test.parquet")
# df = df.sample(frac=1).reset_index(drop=True)



def filter_text(text):
    return ''.join(ch for ch in text if ch in allowed_chars)

def get_most_common_pair(seq):
    counts = defaultdict(int)
    for i in range(len(seq) - 1):
        pair = (seq[i], seq[i+1])
        counts[pair] += 1
    return max(counts, key=counts.get)

def update_seq(seq, pair):
    new_seq = []
    i = 0
    while i < len(seq):
        if i < len(seq) - 1 and (seq[i], seq[i+1]) == pair:

            new_seq.append(''.join(pair))
            i += 2
        else:
            new_seq.append(seq[i])
            i += 1
    return new_seq

def train_bpe(text, num_merges):
    i = 0
    mapping = defaultdict(int)
    # for char in allowed_chars:
    #     mapping[char] = i
    #     i += 1

    for _ in range(num_merges):
        pair = get_most_common_pair(text)
        text = update_seq(text, pair)
        mapping[pair[0]+pair[1]] = i
        i += 1
    return mapping

def encode(text, token_to_id):
    out = []
    current = ""
    for i, char in enumerate(text):
        print(i)
        if current + char in token_to_id:
            print("continuing, ", current + char)
            current += char
        else:
            print("appending, ", current)
            out.append(token_to_id[current])
            current = char
    out.append(token_to_id[current])
    return out

def decode(token_ids, id2tok):
    out = ""
    for i in token_ids:
        out += id2tok[i]
    return out



# with open(r'tokenizer\token_to_id.json', 'w') as f:
#     json.dump(token_to_id, f)

# with open(r'tokenizer\merges.json', 'w') as f:
#     json.dump([list(pair) for pair in merges], f)

text = "hello hero help!"

tok2id = train_bpe(text, num_merges=10)
id2tok = {v: k for k, v in tok2id.items()}

In [120]:
tok2id

defaultdict(int,
            {'he': 0,
             'hel': 1,
             'o ': 2,
             'hell': 3,
             'hello ': 4,
             'hello he': 5,
             'hello her': 6,
             'hello hero ': 7,
             'hello hero hel': 8,
             'hello hero help': 9,
             '': 0,
             'r': 0,
             'p': 0,
             '!': 0})

In [113]:
id2tok

{0: 'he', 1: 'hel', 2: 'o ', 3: 'hell', 4: 'hello '}

In [115]:
for i, char in enumerate("hello hero help!"):
    print(i, char)

0 h
1 e
2 l
3 l
4 o
5  
6 h
7 e
8 r
9 o
10  
11 h
12 e
13 l
14 p
15 !
